### Oil type by origin "US" (general attribution)
Goal: Determine if null values for oil transfer from 
- Phillips Tacoma [all vessels], 
- Kinder Morgan Liquids [atbs]
- Grays Harbour [all vessels]

are legit?


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yaml

file_dir  = '/Users/rmueller/Data/MIDOSS/DeptOfEcology/'
file_name = 'MuellerTrans4-30-20.xlsx'

# Import columns are: (G) Deliverer, (H) Receiver, (P) Product, 
#                     (Q) Quantity in Gallons, (R) Transfer Type (Fueling, Cargo, or Other)', 
#                     (w) DelivererTypeDescription, (x) ReceiverTypeDescription 
df = pd.read_excel(f'{file_dir}{file_name}',sheet_name='Vessel Oil Transfer', 
                   usecols="G,H,P,Q,R,W,X")

# list of names in DOE dataset that are used as "DelivererTypeDescription" for vessel transfers
vessel_names = ['TANK SHIP', 'TANK BARGE', 'TUGBOAT']

# list of terminals that we want to focus on for marine transfers
terminals = [
    'REG Grays Harbor, LLC', 
    'Kinder Morgan Liquids Terminal - Harbor Island', 
    'Phillips 66 Tacoma Terminal'
]

#### Extract cargo data transferred from facility to vessel

In [2]:

# Get all cargo fuel transfers
bool_cargo = df['TransferType']=='Cargo'
cargo_data = df[bool_cargo]
cargo_from_facility = cargo_data[ cargo_data['DelivererTypeDescription']=='Facility' ]
cargo_to_tank   = cargo_from_facility[ cargo_from_facility['ReceiverTypeDescription'] == 'TANK SHIP' ]
cargo_to_barge  = cargo_from_facility[ cargo_from_facility['ReceiverTypeDescription'] == 'TANK BARGE' ] 
cargo_to_tug    = cargo_from_facility[ cargo_from_facility['ReceiverTypeDescription'] == 'TUGBOAT' ]

# reset indices so they increase chronologically from 0 instead of keeping parent index values
cargo_to_tank.reset_index( drop=True, inplace=True )
cargo_to_barge.reset_index( drop=True, inplace=True )
cargo_to_tug.reset_index( drop=True, inplace=True )

In [15]:
greys_origin = df[ df.Deliverer == 'REG Grays Harbor, LLC' ]
kinder_origin = df[ df.Deliverer == 'Kinder Morgan Liquids Terminal - Harbor Island' ]
phillips_origin = df[ df.Deliverer == 'Phillips 66 Tacoma Terminal']

greys_destination = df[ df.Receiver == 'REG Grays Harbor, LLC' ]
kinder_destination = df[ df.Receiver == 'Kinder Morgan Liquids Terminal - Harbor Island' ]
phillips_destination = df[ df.Receiver == 'Phillips 66 Tacoma Terminal']

In [18]:
transfer_data = {'Grays Harbor' :  [sum(greys_origin.TransferQtyInGallon), sum(greys_destination.TransferQtyInGallon)],
        'Kinder Morgan': [sum(kinder_origin.TransferQtyInGallon), sum(kinder_destination.TransferQtyInGallon)],
        'Phillips 66 Tacoma': [sum(phillips_origin.TransferQtyInGallon), sum(phillips_destination.TransferQtyInGallon)]
}

In [19]:
transfer_data

{'Grays Harbor': [0, 21094901],
 'Kinder Morgan': [94446786, 103362000],
 'Phillips 66 Tacoma': [0, 94191800]}

In [20]:
kinder_origin.shape

(148, 7)

In [23]:
kinder_origin[:1]

,Deliverer,Receiver,Product,TransferQtyInGallon,TransferType,DelivererTypeDescription,ReceiverTypeDescription
56,Kinder Morgan Liquids Terminal - Harbor Island,DR. BONNIE W. RAMSEY,BUNKER OIL/HFO,159600,Cargo,Facility,TANK BARGE


In [34]:
kinder_barge = kinder_origin[kinder_origin.ReceiverTypeDescription == 'TANK BARGE']

In [27]:
kinder_barge.shape

(145, 7)

In [29]:
kinder_tug = kinder_origin[kinder_origin.ReceiverTypeDescription == 'TUGBOAT']

In [30]:
kinder_tug.shape

(0, 7)

In [32]:
kinder_tanker = kinder_origin[kinder_origin.ReceiverTypeDescription == 'TANK SHIP']
kinder_tanker.shape

(3, 7)

In [39]:
len(kinder_barge[kinder_barge.Receiver=='DR. BONNIE W. RAMSEY']) + len(kinder_barge[kinder_barge.Receiver ==  'NATHAN SCHMIDT'])

130

In [47]:
kinder_sub = kinder_barge[kinder_barge.Receiver !=  'NATHAN SCHMIDT']
leftover = kinder_sub[kinder_sub.Receiver !=  'DR. BONNIE W. RAMSEY']
leftover

,Deliverer,Receiver,Product,TransferQtyInGallon,TransferType,DelivererTypeDescription,ReceiverTypeDescription
1152,Kinder Morgan Liquids Terminal - Harbor Island,OLYMPIC SPIRIT,BUNKER OIL/HFO,1680000,Cargo,Facility,TANK BARGE
1981,Kinder Morgan Liquids Terminal - Harbor Island,OLYMPIC SPIRIT,CUTTER STOCK,840000,Cargo,Facility,TANK BARGE
1982,Kinder Morgan Liquids Terminal - Harbor Island,OLYMPIC SPIRIT,BUNKER OIL/HFO,840000,Cargo,Facility,TANK BARGE
2941,Kinder Morgan Liquids Terminal - Harbor Island,SHAUNA KAY,DIESEL/MARINE GAS OIL,441000,Cargo,Facility,TANK BARGE
3635,Kinder Morgan Liquids Terminal - Harbor Island,SHAUNA KAY,DIESEL LOW SULPHUR (ULSD),567000,Cargo,Facility,TANK BARGE
3965,Kinder Morgan Liquids Terminal - Harbor Island,SHAUNA KAY,DIESEL LOW SULPHUR (ULSD),567000,Cargo,Facility,TANK BARGE
4495,Kinder Morgan Liquids Terminal - Harbor Island,SHAUNA KAY,DIESEL LOW SULPHUR (ULSD),567000,Cargo,Facility,TANK BARGE
4837,Kinder Morgan Liquids Terminal - Harbor Island,SHAUNA KAY,DIESEL/MARINE GAS OIL,567000,Cargo,Facility,TANK BARGE
5243,Kinder Morgan Liquids Terminal - Harbor Island,SHAUNA KAY,DIESEL LOW SULPHUR (ULSD),273000,Cargo,Facility,TANK BARGE
5496,Kinder Morgan Liquids Terminal - Harbor Island,SHAUNA KAY,DIESEL LOW SULPHUR (ULSD),567000,Cargo,Facility,TANK BARGE
